In [59]:
import tensorflow
import numpy as np
from numpy.linalg import norm
from keras import layers
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import GlobalMaxPooling2D,Flatten, Dense, Dropout
from keras.models import Sequential
from tqdm import tqdm
import os
import cv2
import pickle

In [40]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

In [41]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D(),
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d_1          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling2D)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [42]:
img = cv2.imread("/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset/1636.jpg")
img = cv2.resize(img,(224,224))
img = np.array(img)

In [43]:
img.shape

(224, 224, 3)

In [44]:
# (number_of_image, 224,224,3)
expand_img = np.expand_dims(img, axis=0)

In [45]:
expand_img.shape

(1, 224, 224, 3)

In [46]:
prepro_img = preprocess_input(expand_img)

In [47]:
prepro_img

array([[[[151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         ...,
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ]],

        [[151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         ...,
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ]],

        [[151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         ...,
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ]],

        ...,

        [[151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         ...,


In [48]:
res = model.predict(prepro_img).flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [49]:
normalized = res/norm(res)

In [50]:
normalized.shape

(2048,)

In [51]:
def extract_features(img_path,model):
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224)) 
    img = np.array(img)
    expand_img = np.expand_dims(img, axis=0)
    prepro_img = preprocess_input(expand_img)
    res = model.predict(prepro_img).flatten()
    normalized = res/norm(res)

    return normalized

In [52]:
extract_features("/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset/1636.jpg",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


array([0.00596656, 0.00037371, 0.0081154 , ..., 0.01471429, 0.0150958 ,
       0.02696897], dtype=float32)

In [55]:
filename = []
feature_list = [] 

for file in os.listdir("/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset"):
    if file.endswith(".jpg"):
        filename.append(os.path.join("/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset",file))

In [56]:
filename[0:5]

['/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset/15259.jpg',
 '/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset/44770.jpg',
 '/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset/4217.jpg',
 '/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset/4571.jpg',
 '/Users/tirtarumy/Documents/Data scientist porto/fashion recommendation system/Dataset/9055.jpg']

In [57]:
for file in tqdm(filename):
    feature_list.append(extract_features(file,model))

  0%|          | 0/2906 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 901ms/step


  0%|          | 1/2906 [00:01<1:17:47,  1.61s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


  0%|          | 2/2906 [00:01<36:41,  1.32it/s]  

...

100%|█████████▉| 2902/2906 [04:00<00:00, 14.70it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


100%|█████████▉| 2904/2906 [04:00<00:00, 13.58it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


100%|██████████| 2906/2906 [04:01<00:00, 12.05it/s]


In [60]:
pickle.dump(feature_list,open("feature_vector.pkl","wb"))
pickle.dump(filename,open("filenames.pkl","wb"))